In [1]:
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader


# Datasetを作成
train_imgs = ImageFolder(
    'taco_and_burrito/train/', transform=transforms.Compose([transforms.RandomCrop(224), transforms.ToTensor()])
)
test_imgs = ImageFolder(
    'taco_and_burrito/test/', transform=transforms.Compose([transforms.CenterCrop(224), transforms.ToTensor()])
)
# DataLoaderを作成
train_loader = DataLoader(
    train_imgs, batch_size=32, shuffle=True
)
test_loader = DataLoader(
    test_imgs, batch_size=32, shuffle=True
)

In [2]:
# ラベルを確認する
print(train_imgs.classes)
print(train_imgs.class_to_idx)

['burrito', 'taco']
{'burrito': 0, 'taco': 1}


In [3]:
from torch import nn
from torchvision import models


net = models.resnet18(pretrained=True)
for p in net.parameters():
    p.requires_grad = False
fc_input_dim = net.fc.in_features
net.fc = nn.Linear(fc_input_dim, 2)

In [4]:
import torch


def eval_net(net, data_loader):
    net.eval()
    ys = []
    ypreds = []
    for x, y in data_loader:
        x = V(x, volatile=True)
        y = V(y, volatile=True)
        _, y_pred = net(x).max(1)
        ys.append(y)
        ypreds.append(y_pred)
    ys = torch.cat(ys)
    ypreds = torch.cat(ypreds)
    acc = (ys == ypreds).float().sum() / len(ys)
    return acc.data[0]

In [5]:
from torch import optim
from torch.autograd import Variable as V
from tqdm import tqdm


def train_net(net, train_loader, test_loader, only_fc=True, optimizer_cls=optim.Adam, loss_fn=nn.CrossEntropyLoss(), n_iter=10):
    train_losses = []
    train_acc = []
    val_acc = []
    if only_fc:
        optimizer = optimizer_cls(net.fc.parameters())
    else:
        optimizer = optimizer_cls(net.parameters())
    for epoch in range(n_iter):
        running_loss = 0.0
        net.train()
        n = 0
        n_acc = 0
        for i, (x, y) in tqdm(enumerate(train_loader), total=len(train_loader)):
            xx = V(x)
            yy = V(y)
            h = net(xx)
            loss = loss_fn(h, yy)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.data[0]
            n += len(xx)
            _, y_pred = h.max(1)
            n_acc += (yy == y_pred).float().sum().data[0]
        train_losses.append(running_loss / i)
        train_acc.append(n_acc / n)
        val_acc.append(eval_net(net, test_loader))
        print(epoch, train_losses[-1], train_acc[-1], val_acc[-1], flush=True)

In [6]:
train_net(net, train_loader, test_loader)

100%|██████████| 23/23 [00:47<00:00,  2.08s/it]


0 0.675819066437808 0.6095505617977528 0.800000011920929


100%|██████████| 23/23 [00:48<00:00,  2.10s/it]


1 0.5339952084151182 0.7570224719101124 0.8666666746139526


100%|██████████| 23/23 [00:49<00:00,  2.14s/it]


...

7 0.363796281543645 0.8567415730337079 0.7833333611488342


100%|██████████| 23/23 [00:49<00:00,  2.17s/it]


8 0.3537446450103413 0.8721910112359551 0.8999999761581421


100%|██████████| 23/23 [00:48<00:00,  2.13s/it]


9 0.3585528602654284 0.848314606741573 0.8999999761581421


In [7]:
class FlattenLayer(nn.Module):
    def forward(self, x):
        sizes = x.size()
        return x.view(sizes[0], -1)

    
conv_net = nn.Sequential(
    nn.Conv2d(3, 32, 5),
    nn.MaxPool2d(2),
    nn.ReLU(),
    nn.BatchNorm2d(32),
    nn.Conv2d(32, 64, 5),
    nn.MaxPool2d(2),
    nn.ReLU(),
    nn.BatchNorm2d(64),
    nn.Conv2d(64, 128, 5),
    nn.MaxPool2d(2),
    nn.ReLU(),
    nn.BatchNorm2d(128),
    FlattenLayer()
)

test_input = V(torch.ones(1, 3, 224, 224))
conv_output_size = conv_net(test_input).size()[-1]
net = nn.Sequential(
    conv_net,
    nn.Linear(conv_output_size, 2)
)

In [8]:
train_net(net, train_loader, test_loader, n_iter=10, only_fc=False)

100%|██████████| 23/23 [01:15<00:00,  3.28s/it]


0 2.388649122281508 0.5365168539325843 0.4833333194255829


100%|██████████| 23/23 [01:15<00:00,  3.29s/it]


1 2.5478381568735298 0.5898876404494382 0.5333333611488342


100%|██████████| 23/23 [01:16<00:00,  3.33s/it]


...

7 2.235762216828086 0.6502808988764045 0.6333333253860474


100%|██████████| 23/23 [01:16<00:00,  3.35s/it]


8 1.8493617556311868 0.6797752808988764 0.699999988079071


100%|██████████| 23/23 [01:18<00:00,  3.40s/it]


9 2.2664984085343103 0.6278089887640449 0.5833333134651184
